In [1]:
import hashlib
from collections import namedtuple

In [2]:
cell = namedtuple('Cell', 'x y code path')


class SecureVault:
    
    dir_dict = {'U': (0, -1), 'D': (0, +1), 'R': (1, 0), 'L': (-1, 0)}
    
    def __init__(self, movements):
        self.movements = movements
    
    @staticmethod
    def get_md5hash(to_hash):
        return hashlib.md5(bytes('{}'.format(to_hash), encoding='ascii')).hexdigest().lower()

    @staticmethod
    def is_valid(x, y):
        if 0 <= x < 4 and 0 <= y < 4:
            return True
        return False
    
    @staticmethod
    def get_directions(hashed):
        """U for up, D for down, L for left, and R for right. 
        Any b, c, d, e, or f means that the corresponding door is open; 
        any other character (any number or a) means that the corresponding 
        door is closed and locked
        """
        directions = []
        for direction, character in zip('UDLR', hashed[:4]):
            if character in 'bcdef':
                directions.append(direction)
        return directions
    
    def shortest_path(self):
        if len(self.movements) == 0:
            return False
        mov = self.movements.pop(0)
        if mov.x == 3 and mov.y == 3:
            return mov.path
        for direction in self.get_directions(self.get_md5hash(mov.code)):
            x_, y_ = self.dir_dict[direction]
            if self.is_valid(x=mov.x + x_, y=mov.y + y_):
                self.movements.append(
                    cell(
                        x=mov.x + x_, 
                        y=mov.y + y_, 
                        code=mov.code + direction, 
                        path=mov.path + direction
                    )
                )
        return self.shortest_path()
    
    def longest_path(self):
        longest_path = 0
        while len(self.movements) > 0:
            mov = self.movements.pop(0)
            if mov.x == 3 and mov.y == 3:
                longest_path = len(mov.path) if len(mov.path) > longest_path else longest_path
                continue
            for direction in self.get_directions(self.get_md5hash(mov.code)):
                x_, y_ = self.dir_dict[direction]
                if self.is_valid(x=mov.x + x_, y=mov.y + y_):
                    self.movements.append(
                        cell(
                            x=mov.x + x_, 
                            y=mov.y + y_, 
                            code=mov.code + direction, 
                            path=mov.path + direction
                        )
                    )
        return longest_path

In [3]:
# Tests
movements = [cell(x=0, y=0, code='ihgpwlah', path='')]
assert SecureVault(movements).shortest_path() == 'DDRRRD'
movements = [cell(x=0, y=0, code='kglvqrro', path='')]
assert SecureVault(movements).shortest_path() == 'DDUDRLRRUDRD'
movements = [cell(x=0, y=0, code='ulqzkmiv', path='')]
assert SecureVault(movements).shortest_path() == 'DRURDRUDDLLDLUURRDULRLDUUDDDRR'

In [4]:
movements = [cell(x=0, y=0, code='dmypynyp', path='')]
SecureVault(movements).shortest_path()

'RDRDUDLRDR'

In [5]:
# Tests
movements = [cell(x=0, y=0, code='ihgpwlah', path='')]
assert SecureVault(movements).longest_path() == 370
movements = [cell(x=0, y=0, code='kglvqrro', path='')]
assert SecureVault(movements).longest_path() == 492
movements = [cell(x=0, y=0, code='ulqzkmiv', path='')]
assert SecureVault(movements).longest_path() == 830

In [6]:
movements = [cell(x=0, y=0, code='dmypynyp', path='')]
SecureVault(movements).longest_path()

386